# DS204 — Final Project: Analysis for Data Science

👥 **Team:** _Aaron, Andres, Lucas, Melissa_

📦 **Dataset:** _<Students Performance in Exams / [Source link](https://www.kaggle.com/datasets/spscientist/students-performance-in-exams)>_

🗂️ **Repo:** _<[GitHub URL](https://github.com/Lucasyuki01/DS204-FinalProject)>_

> **Goal:** Produce a complete data quality analysis, identify 2–3 meaningful associations (Pearson, Spearman, Kendall), fix data quality issues programmatically, suggest a prediction approach (focus on linear regression), and export the cleaned dataset.

## 0. Reproducibility & Environment
Record package versions used so the notebook is fully reproducible.

In [ ]:
# %% [reproducibility]
import sys, platform
import pandas as pd, numpy as np
import sklearn, statsmodels
import matplotlib, seaborn
try:
    import ydata_profiling
except Exception:
    ydata_profiling = None
from datetime import datetime

print({
    'python': sys.version.split()[0],
    'platform': platform.platform(),
    'pandas': pd.__version__,
    'numpy': np.__version__,
    'scikit_learn': sklearn.__version__,
    'statsmodels': statsmodels.__version__,
    'matplotlib': matplotlib.__version__,
    'seaborn': seaborn.__version__,
    'ydata_profiling': getattr(ydata_profiling, '__version__', 'not installed'),
    'timestamp': datetime.utcnow().isoformat()+'Z'
})

## 1. Setup & Data Loading
Use a relative path. Keep raw data unchanged; write cleaned output to `data/Dataset-Fixed.csv`.

## 2. Quick Metadata & Basic Checks
High-level overview for context and to feed the DQ report.

## 3. Data Quality Report (ydata_profiling)
Generates `profile.html` to be submitted. If you prefer DataPrep, swap accordingly.

## 4. Correlations (Pearson, Spearman, Kendall)
Compute correlation matrices programmatically to cross-check the profiling tool.

## 5. Candidate Associations (2–3 pairs)
Select pairs based on domain logic + coefficients. Add short rationale and plots.

## 6. Data Quality Issues & Fixes
Show evidence (counts/plots) and apply fixes with **clear rationale**. Keep raw intact.

## 7. Export Cleaned Dataset
Save the cleaned data as `Dataset-Fixed.csv` as required.

## 8. Modeling Suggestion (Linear Regression)
If assumptions hold, fit a simple linear regression. Otherwise, suggest a better alternative and justify.

## 9. Conclusions
- Summary of associations and their interpretation.
- Data quality issues and fixes (with % affected).
- Recommended model and rationale.
- Limitations and next steps.

## Appendix
### A. Encoding Categorical Variables (if needed)